In [1]:
import sys
import os
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
from configs import configs
import importlib
from generator.generator import GenThreadSafe, SemisupWrapper
from generator.batch_fetcher import BatchFetcher
if __name__ == "__main__":
    sys.path.append("../../")
from configs import configs
label_maker_module = importlib.import_module("notebooks_src.load_data.label_maker." + configs["label_maker_name"] + ".make_labels")

importing Jupyter notebook from configs.ipynb
importing Jupyter notebook from generator/generator.ipynb
importing Jupyter notebook from generator/batch_fetcher.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/label_maker/yolo/make_labels.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/label_maker/yolo/label_util.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/configs.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/label_maker/box_processing/coord_conversion.ipynb


In [2]:
create_labelmaker_fxn = label_maker_module.create_labelmaker_fxn

In [3]:
def make_batch_fetcher(typ="tr", num_examples=-1,data_name="climate", data_path=configs["data_file"]):
    data_module = importlib.import_module("notebooks_src.load_data.datasets." + data_name+".get_data")
    ims, labels = data_module.get_data(type_=typ, data_file=data_path)
    return BatchFetcher(ims,labels,num_examples=num_examples)

In [4]:
def get_generator(typ, data_path=configs["data_file"],
                  data_name=configs["data_name"], 
                  batch_size=None,num_ims=-1, 
                  mode="supervised" ):
    

    num_ims = configs["num_"+ typ+"_ims"]
    batch_fetcher = make_batch_fetcher(num_examples=num_ims,typ=typ,
                                       data_name=data_name, data_path=data_path)
    
    
    all_labels = batch_fetcher.labels
    make_label_fn = create_labelmaker_fxn(all_labels)
    
    if batch_size is None:
        batch_size = configs["batch_size"]
    generator = GenThreadSafe(batch_fetcher, 
                              shape=configs["input_shape"],#configs["input_shape"],
                              batch_size = batch_size,
                              typ=typ, 
                              tf_mode=True, 
                              num_ex = num_ims, 
                              make_label_fxn=make_label_fn) 
    
    if mode == "semi_supervised":
        generator = SemisupWrapper(generator)
        
    return generator

In [5]:
bf = make_batch_fetcher()

importing Jupyter notebook from ../../notebooks_src/load_data/datasets/climate/get_data.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/datasets/climate/util.ipynb


In [14]:
bf.labels

[[array([ 164,  244, 1037, 1117,    3], dtype=int32)],
 [array([ 162,  242, 1054, 1134,    3], dtype=int32),
  array([117, 197, 560, 640,   3], dtype=int32)],
 [array([ 159,  239, 1072, 1152,    3], dtype=int32),
  array([118, 198, 566, 646,   3], dtype=int32)],
 [array([300, 320, 371, 391,   2], dtype=int32),
  array([ 154,  234, 1089, 1169,    3], dtype=int32),
  array([122, 202, 576, 656,   3], dtype=int32)],
 [array([308, 318, 376, 386,   2], dtype=int32),
  array([ 152,  232, 1108, 1188,    3], dtype=int32),
  array([ 97, 177, 610, 690,   3], dtype=int32)],
 [array([295, 329, 360, 394,   2], dtype=int32),
  array([147, 227, -30,  50,   3], dtype=int32),
  array([ 91, 171, 620, 700,   3], dtype=int32)],
 [array([298, 326, 361, 389,   2], dtype=int32),
  array([145, 225, -14,  66,   3], dtype=int32),
  array([ 86, 166, 630, 710,   3], dtype=int32)],
 [array([296, 328, 355, 387,   2], dtype=int32)],
 [array([294, 328, 351, 385,   2], dtype=int32)],
 [array([295, 327, 348, 380,   2], 

In [5]:
#gen=get_generator("tr")

importing Jupyter notebook from ../../notebooks_src/load_data/datasets/climate/get_data.ipynb
importing Jupyter notebook from ../../notebooks_src/load_data/datasets/climate/util.ipynb
